# Bags of Popcorn

In [1]:
import pandas as pd

train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

## Cleaning the Data

In [11]:
import nltk
nltk.download() 

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [12]:
from bs4 import BeautifulSoup    
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

def review_to_words( raw_review ):
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Hash numbers and remove non-letters 
    hashed_review_text = re.sub('\d', '#', review_text) 
    lim_hashed_review_text = re.sub("#####+", "#####", hashed_review_text)
    letters_only = re.sub("[^a-zA-Z#]", " ", lim_hashed_review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words and lemmatize
    meaningful_words = [WordNetLemmatizer().lemmatize(w) for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [82]:
# checking the clean
review_to_words("56567 flowers /liking likes like 6789098765678 ///")

u'##### flower liking like like #####'

In [14]:
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size
print "Cleaning and parsing the training set movie reviews...\n"
clean_train_reviews = []
for i in xrange( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )                                                                    
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [56]:
num__imdb_reviews = train_imdb["review"].size
print "Cleaning and parsing the training set movie reviews...\n"
clean_train_imdb_reviews = []
for i in xrange( 0, num__imdb_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num__imdb_reviews )      
    clean_train_imdb_reviews.append(review_to_words( train_imdb["review"][i]))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



## Tf-Idf Vectorizer

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

train_data_features_tfidf = vectorizer.fit_transform(clean_train_reviews)
train_data_features_tfidf = train_data_features_tfidf.toarray()

In [ ]:
train_imdb_data_features_tfidf = vectorizer.fit_transform(clean_train_imdb_reviews)
train_imdb_data_features_tfidf = train_imdb_data_features_tfidf.toarray()

### Logistic Regression for Tf-Idf Vec IMDB data

In [58]:
from sklearn.linear_model import LogisticRegression

log_regression_tfidf = LogisticRegression()
log_regression_tfidf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [62]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': (0.25, 0.5, 1.0), 'penalty': ('l1', 'l2')}
grid_search = GridSearchCV(log_regression_tfidf, param_grid, n_jobs=-1, scoring="roc_auc")
print("Performing grid search...")

grid_search.fit(train_imdb_data_features_tfidf, train_imdb.sentiment)
print("Best CV score: %0.3f" % grid_search.best_score_)
print("Best parameters set:", grid_search.best_estimator_.get_params())

Performing grid search...
Best CV score: 0.936
('Best parameters set:', {'warm_start': False, 'C': 1.0, 'n_jobs': None, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'multi_class': 'warn', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'warn', 'class_weight': None})


In [63]:
from sklearn.metrics import classification_report
print test_imdb.shape

# Create an empty list and append the clean reviews one by one
num__imdb_reviews = test_imdb["review"].size
print "Cleaning and parsing the training set movie reviews...\n"
clean_test_imdb_reviews = []
for i in xrange( 0, num__imdb_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num__imdb_reviews )      
    clean_test_imdb_reviews.append(review_to_words( test_imdb["review"][i]))

# Get a bag of words for the test set, and convert to a numpy array
test_imdb_data_features_tfidf = vectorizer.transform(clean_test_imdb_reviews)
test_imdb_data_features_tfidf = test_imdb_data_features_tfidf.toarray()
    
print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(test_imdb_data_features_tfidf, test_imdb.sentiment))
print("Classification Report Test Data")
print(classification_report(test_imdb.sentiment, grid_search.best_estimator_.predict(test_imdb_data_features_tfidf)))

(25000, 3)
Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000

Test score with best_estimator_: 0.877
Classification Report Test Data
              precision    recall  f1-score   support

           0       0.88      0.87      0.88     12500
           1       0.87      0.89      0.88     12500

   micro avg       0.88      0.88      0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88 

### Logistic Regression for Tf-Idf Vec Kaggle data

In [65]:
log_regression = LogisticRegression()
log_regression.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [66]:
param_grid = {'C': (0.25, 0.5, 1.0), 'penalty': ('l1', 'l2')}
grid_search_tfidf = GridSearchCV(log_regression, param_grid, n_jobs=-1, cv=5, scoring="roc_auc")
print("Performing grid search...")

grid_search_tfidf.fit(train_data_features_tfidf, train.sentiment)
print("Best CV score: %0.3f" % grid_search_tfidf.best_score_)
print("Best parameters set:", grid_search_tfidf.best_estimator_.get_params())

Performing grid search...
Best CV score: 0.951
('Best parameters set:', {'warm_start': False, 'C': 1.0, 'n_jobs': None, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l2', 'multi_class': 'warn', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'warn', 'class_weight': None})


In [71]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features_tfidf = vectorizer.transform(clean_test_reviews)
test_data_features_tfidf = test_data_features_tfidf.toarray()

# Use the random forest to make sentiment label predictions
result = grid_search_tfidf.best_estimator_.predict(test_data_features_tfidf)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Tfidf_LinReg_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



### Random Forest for Tf-Idf Vec

In [69]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
forest = forest.fit(train_data_features, train["sentiment"] )

Training the random forest...


### Saving the results of Random Forest

In [103]:
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features_tfidf)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Tfidf_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



### MLP for Tf-Idf Vec

In [114]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
mlp_tfidf = mlp.fit(train_data_features, train["sentiment"])

# Make sentiment label predictions
result = mlp_tfidf.predict(test_data_features_tfidf)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Tfidf_mlp_model.csv", index=False, quoting=3 )

## Bag of Words Vectorizer

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features_bw = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features_bw = train_data_features_bw.toarray()

In [97]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'abandoned', u'abc', u'ability', u'able', u'abraham', u'abrupt', u'absence', u'absent', u'absolute', u'absolutely', u'absurd', u'absurdity', u'abuse', u'abused', u'abusive', u'abysmal', u'academy', u'accent', u'accept', u'acceptable', u'accepted', u'accepts', u'access', u'accident', u'accidentally', u'acclaimed', u'accompanied', u'accomplish', u'accomplished', u'according', u'account', u'accuracy', u'accurate', u'accused', u'ace', u'achieve', u'achieved', u'achievement', u'acid', u'across', u'act', u'acted', u'acting', u'action', u'active', u'activity', u'actor', u'actress', u'actual', u'actually', u'ad', u'adam', u'adaptation', u'adapted', u'add', u'added', u'addict', u'addicted', u'addiction', u'adding', u'addition', u'additional', u'address', u'adequate', u'admirable', u'admire', u'admit', u'admittedly', u'adolescent', u'adopted', u'adorable', u'adult', u'advance', u'advanced', u'advantage', u'adventure', u'advertising', u'advice', u'advise', u'affair', u'affect', u'affected', u'a

In [98]:
#print the counts of each word in the vocabulary
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features_bw, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print count, tag

187 abandoned
125 abc
562 ability
1259 able
92 abraham
73 abrupt
118 absence
83 absent
352 absolute
1485 absolutely
306 absurd
86 absurdity
202 abuse
77 abused
91 abusive
98 abysmal
298 academy
688 accent
300 accept
130 acceptable
144 accepted
74 accepts
92 access
337 accident
200 accidentally
75 acclaimed
88 accompanied
77 accomplish
124 accomplished
295 according
243 account
82 accuracy
284 accurate
123 accused
73 ace
179 achieve
139 achieved
169 achievement
91 acid
971 across
1645 act
658 acted
6491 acting
3665 action
76 active
146 activity
6875 actor
1588 actress
793 actual
4237 actually
188 ad
400 adam
533 adaptation
154 adapted
1147 add
439 added
82 addict
78 addicted
72 addiction
166 adding
371 addition
74 additional
103 address
113 adequate
73 admirable
124 admire
621 admit
134 admittedly
94 adolescent
75 adopted
101 adorable
886 adult
140 advance
90 advanced
170 advantage
714 adventure
91 advertising
262 advice
90 advise
417 affair
155 affect
113 affected
105 affection
104 aff

193 darkness
73 darn
83 darren
507 date
267 dated
107 dating
1312 daughter
117 dave
1025 david
354 davis
140 davy
147 dawn
133 dawson
4013 day
792 de
1881 dead
201 deadly
82 deaf
975 deal
99 dealer
284 dealing
137 dealt
189 dean
147 dear
2113 death
78 debate
81 debt
266 debut
432 decade
82 deceased
1157 decent
482 decide
705 decided
546 decides
345 decision
99 dedicated
82 dee
97 deed
654 deep
177 deeper
320 deeply
127 defeat
95 defend
92 defense
87 defined
117 definite
1580 definitely
90 definition
256 degree
97 del
72 deleted
108 deliberately
179 delight
274 delightful
328 deliver
243 delivered
94 delivering
356 delivers
188 delivery
203 demand
89 demented
83 demise
375 demon
82 deniro
176 dennis
168 dentist
141 denzel
223 department
210 depicted
82 depicting
222 depiction
90 depicts
99 depressed
226 depressing
129 depression
570 depth
86 der
78 deranged
151 derek
91 descent
330 describe
233 described
102 describes
195 description
272 desert
73 deserted
287 deserve
291 deserved
591 d

137 health
733 hear
1111 heard
238 hearing
78 hears
1463 heart
74 heartbreaking
225 hearted
75 heartfelt
133 heat
355 heaven
180 heavily
502 heavy
222 heck
83 heel
155 height
76 heist
391 held
152 helen
123 helicopter
1032 hell
90 hello
2255 help
324 helped
179 helping
155 hence
83 henchman
407 henry
72 hepburn
1388 hero
131 heroic
315 heroine
136 heston
409 hey
342 hidden
275 hide
103 hideous
144 hiding
2174 high
289 higher
106 highest
327 highlight
1147 highly
973 hilarious
86 hilariously
395 hill
250 hint
190 hip
84 hippie
84 hippy
191 hire
188 hired
72 historic
406 historical
86 historically
1337 history
1360 hit
210 hitchcock
308 hitler
137 hitting
74 hk
137 ho
188 hoffman
845 hold
209 holding
534 hole
180 holiday
113 hollow
107 holly
1911 hollywood
163 holmes
114 holy
153 homage
1939 home
140 homeless
104 homer
108 homosexual
481 honest
453 honestly
99 honesty
191 hong
191 honor
189 hood
127 hook
139 hooked
101 hop
1720 hope
144 hoped
214 hopefully
82 hopeless
74 hopelessly
407 h

154 musician
136 muslim
3249 must
98 mutant
97 myers
404 mysterious
955 mystery
91 myth
169 nail
218 naive
437 naked
1995 name
799 named
96 namely
228 nancy
197 narration
448 narrative
129 narrator
342 nasty
82 nathan
249 nation
258 national
304 native
463 natural
267 naturally
723 nature
74 naughty
162 navy
304 nazi
145 nd
824 near
109 nearby
815 nearly
149 neat
180 necessarily
324 necessary
164 neck
182 ned
2648 need
683 needed
198 needle
377 negative
228 neighbor
158 neighborhood
134 neil
537 neither
153 nelson
79 nemesis
112 neo
99 nephew
115 nerd
119 nerve
97 nervous
73 net
233 network
6484 never
236 nevertheless
4310 new
77 newcomer
89 newly
79 newman
331 news
139 newspaper
1716 next
2012 nice
299 nicely
92 nicholas
113 nicholson
296 nick
82 nicole
73 niece
2293 night
426 nightmare
163 nine
88 ninety
128 ninja
100 niro
158 noble
462 nobody
102 nod
419 noir
214 noise
75 nolan
222 nominated
160 nomination
898 non
1032 none
163 nonetheless
288 nonsense
82 nonsensical
77 norm
458 nor

127 seller
129 selling
211 semi
231 send
139 sends
79 sens
2325 sense
101 senseless
89 sensibility
181 sensitive
395 sent
156 sentence
109 sentiment
146 sentimental
79 sentinel
165 separate
93 september
1042 sequel
1603 sequence
204 serf
85 sergeant
446 serial
3415 series
989 serious
1002 seriously
136 servant
167 serve
158 served
258 service
87 serving
85 session
3307 set
809 setting
125 settle
74 setup
360 seven
129 seventy
1420 several
98 severe
73 severely
1712 sex
710 sexual
152 sexuality
134 sexually
445 sexy
103 sh
96 shade
298 shadow
127 shake
297 shakespeare
80 shaky
133 shall
259 shallow
674 shame
81 shanghai
188 shape
439 share
75 shared
183 shark
78 sharon
209 sharp
100 shaw
92 shed
247 sheer
80 sheet
140 shelf
72 shell
95 shelley
248 sheriff
107 shift
258 shine
129 shining
426 ship
103 shirley
161 shirt
431 shock
209 shocked
339 shocking
75 shoddy
175 shoe
602 shoot
511 shooting
84 shootout
290 shop
2013 short
77 shortcoming
122 shortly
2998 shot
153 shoulder
8603 show
128

7095 watch
307 watchable
2236 watched
75 watcher
4605 watching
655 water
86 watson
266 wave
72 wax
8830 way
243 wayne
761 weak
99 weakest
147 weakness
109 wealth
153 wealthy
316 weapon
352 wear
326 wearing
120 web
126 website
321 wedding
662 week
216 weekend
142 weight
664 weird
226 welcome
10782 well
251 welles
98 wendy
1463 went
331 werewolf
121 wes
476 west
763 western
98 wet
131 whale
732 whatever
85 whats
319 whatsoever
85 wheel
270 whenever
149 whereas
856 whether
280 whilst
1546 white
202 whoever
3078 whole
82 wholly
84 whoopi
986 whose
118 wicked
281 wide
87 widely
155 widmark
112 widow
2245 wife
82 wig
441 wild
72 wilder
80 wilderness
77 wildly
596 william
344 williams
85 willie
320 willing
92 willis
200 wilson
655 win
376 wind
322 window
246 wing
271 winner
353 winning
242 winter
95 wire
89 wisdom
361 wise
1111 wish
95 wished
85 wishing
268 wit
414 witch
832 within
3267 without
299 witness
88 witnessed
273 witty
122 wizard
156 wolf
4624 woman
1163 wonder
126 wondered
1656 won

### Logistic Regression for BWV

In [76]:
grid_search_bw = GridSearchCV(log_regression, param_grid, n_jobs=-1, cv=5, scoring="roc_auc")
print("Performing grid search...")

grid_search_bw.fit(train_data_features_bw, train.sentiment)
print("Best CV score: %0.3f" % grid_search_bw.best_score_)
print("Best parameters set:", grid_search_bw.best_estimator_.get_params())

Performing grid search...
Best CV score: 0.942
('Best parameters set:', {'warm_start': False, 'C': 0.25, 'n_jobs': None, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'multi_class': 'warn', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'warn', 'class_weight': None})


In [77]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features_bw = vectorizer.transform(clean_test_reviews)
test_data_features_bw = test_data_features_bw.toarray()

# Use the random forest to make sentiment label predictions
result = grid_search_bw.best_estimator_.predict(test_data_features_bw)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_LinReg_model.csv", index=False, quoting=3 )

### Random Forest for BWV

In [99]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
forest = forest.fit(train_data_features, train["sentiment"] )

Training the random forest...


### Saving the results of Random Forest

In [100]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



### MLP for Bag of Words

In [120]:
mlp_bw = mlp.fit(train_data_features_bw, train["sentiment"])

# Make sentiment label predictions
result = mlp_bw.predict(test_data_features_bw)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_mlp_model.csv", index=False, quoting=3 )

## Downloading unlabled data for Word2Vec

In [79]:
import pandas as pd

# Read data from unlabeledTrainData
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



### Cleaning the data for Word2Vec

In [86]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Hash numbers and remove non-letters 
    hashed_review_text = re.sub('\d', '#', review_text) 
    lim_hashed_review_text = re.sub("#####+", "#####", hashed_review_text)
    letters_only = re.sub("[^a-zA-Z#]", " ", lim_hashed_review_text) 
    #
    # 3. Convert words to lower case and split them
    words = letters_only.lower().split()
    # 
    # 4. Lemmatize
    words = [WordNetLemmatizer().lemmatize(w) for w in words]   
    #
    # 5. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [87]:
# cheking the cleaner
print train['review'][1]
a=review_to_wordlist(train['review'][1])
print a

"\"The Classic War of the Worlds\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \"critics\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \"critics\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \"critics\" perceive to be its shortcomings."
[u'the', u'classic', u'war', u'of', u'the', u'world

### Dividing the data into sentences

In [88]:
# Download the punkt tokenizer for sentence splitting
import nltk.data 

# Load the punkt tokenizer
tokenizer = nltk.data.load('/Users/Bai/nltk_data/tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences

    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [89]:
sentences = []  # Initialize an empty list of sentences
print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/Users/Bai/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:273: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/Bai/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/Bai/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/Bai/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/Bai/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requ

In [90]:
#cheking the len and sen0
print len(sentences)
print sentences[1]

795538
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'wa', u'really', u'cool', u'in', u'the', u'eighty', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


### Creating and Saving Word2Vec model

In [91]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-06-05 14:23:57,273 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-05 14:23:57,293 : INFO : collecting all words and their counts
2019-06-05 14:23:57,297 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-05 14:23:57,484 : INFO : PROGRESS: at sentence #10000, processed 227242 words, keeping 16049 word types


Training model...


2019-06-05 14:23:57,756 : INFO : PROGRESS: at sentence #20000, processed 454580 words, keeping 22412 word types
2019-06-05 14:23:57,908 : INFO : PROGRESS: at sentence #30000, processed 675279 words, keeping 26966 word types
2019-06-05 14:23:58,055 : INFO : PROGRESS: at sentence #40000, processed 903019 words, keeping 30813 word types
2019-06-05 14:23:58,198 : INFO : PROGRESS: at sentence #50000, processed 1123513 words, keeping 33853 word types
2019-06-05 14:23:58,357 : INFO : PROGRESS: at sentence #60000, processed 1346275 words, keeping 36516 word types
2019-06-05 14:23:58,587 : INFO : PROGRESS: at sentence #70000, processed 1570750 words, keeping 38854 word types
2019-06-05 14:23:58,758 : INFO : PROGRESS: at sentence #80000, processed 1791260 words, keeping 41015 word types
2019-06-05 14:23:58,914 : INFO : PROGRESS: at sentence #90000, processed 2016735 words, keeping 43205 word types
2019-06-05 14:23:59,164 : INFO : PROGRESS: at sentence #100000, processed 2239910 words, keeping 45

2019-06-05 14:24:10,714 : INFO : PROGRESS: at sentence #740000, processed 16649327 words, keeping 109934 word types
2019-06-05 14:24:10,874 : INFO : PROGRESS: at sentence #750000, processed 16868987 words, keeping 110521 word types
2019-06-05 14:24:11,013 : INFO : PROGRESS: at sentence #760000, processed 17089664 words, keeping 111110 word types
2019-06-05 14:24:11,163 : INFO : PROGRESS: at sentence #770000, processed 17318152 words, keeping 111848 word types
2019-06-05 14:24:11,327 : INFO : PROGRESS: at sentence #780000, processed 17549660 words, keeping 112521 word types
2019-06-05 14:24:11,485 : INFO : PROGRESS: at sentence #790000, processed 17777980 words, keeping 113169 word types
2019-06-05 14:24:11,572 : INFO : collected 113595 word types from a corpus of 17901783 raw words and 795538 sentences
2019-06-05 14:24:11,574 : INFO : Loading a fresh vocabulary
2019-06-05 14:24:11,876 : INFO : effective_min_count=40 retains 14983 unique words (13% of original 113595, drops 98612)
2019-

2019-06-05 14:25:07,464 : INFO : EPOCH 2 - PROGRESS: at 94.49% examples, 442483 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:25:08,465 : INFO : EPOCH 2 - PROGRESS: at 97.93% examples, 442491 words/s, in_qsize 8, out_qsize 0
2019-06-05 14:25:09,161 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-05 14:25:09,176 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-05 14:25:09,180 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-05 14:25:09,184 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-06-05 14:25:09,185 : INFO : EPOCH - 2 : training on 17901783 raw words (12819318 effective words) took 29.1s, 440724 effective words/s
2019-06-05 14:25:10,212 : INFO : EPOCH 3 - PROGRESS: at 2.55% examples, 323445 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:25:11,224 : INFO : EPOCH 3 - PROGRESS: at 6.30% examples, 397805 words/s, in_qsize 8, out_qsize 0
2019-06-05 14:25:12,243 : INFO : EPOCH 3 - PRO

2019-06-05 14:26:09,189 : INFO : EPOCH 5 - PROGRESS: at 36.69% examples, 514798 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:10,199 : INFO : EPOCH 5 - PROGRESS: at 40.74% examples, 514990 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:11,224 : INFO : EPOCH 5 - PROGRESS: at 44.92% examples, 515715 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:12,241 : INFO : EPOCH 5 - PROGRESS: at 48.98% examples, 516038 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:13,257 : INFO : EPOCH 5 - PROGRESS: at 53.07% examples, 515901 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:14,269 : INFO : EPOCH 5 - PROGRESS: at 57.15% examples, 516295 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:15,274 : INFO : EPOCH 5 - PROGRESS: at 61.16% examples, 516475 words/s, in_qsize 8, out_qsize 0
2019-06-05 14:26:16,290 : INFO : EPOCH 5 - PROGRESS: at 65.18% examples, 515917 words/s, in_qsize 7, out_qsize 0
2019-06-05 14:26:17,308 : INFO : EPOCH 5 - PROGRESS: at 69.18% examples, 515301 words/s, in_qsiz

In [92]:
print model.doesnt_match("man woman child kitchen".split())
print model.doesnt_match("france england germany berlin".split())
print model.doesnt_match("paris berlin london austria".split())

kitchen
berlin
paris


/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  
/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [93]:
print model.most_similar("man")
print model.most_similar("queen")

[(u'lad', 0.5136903524398804), (u'lady', 0.5087424516677856), (u'millionaire', 0.49499011039733887), (u'businessman', 0.493978351354599), (u'person', 0.493786096572876), (u'men', 0.480119526386261), (u'woman', 0.4776189923286438), (u'chap', 0.47036212682724), (u'loner', 0.46733981370925903), (u'widow', 0.4560745358467102)]
[(u'princess', 0.626689612865448), (u'bombshell', 0.5967222452163696), (u'maria', 0.5821893215179443), (u'stepmother', 0.5769131183624268), (u'maid', 0.5767726898193359), (u'belle', 0.5742414593696594), (u'victoria', 0.5602350234985352), (u'blonde', 0.5532236099243164), (u'bride', 0.548600435256958), (u'latifah', 0.5465130805969238)]


/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


### Loading Word2Vec model

In [94]:
from gensim.models import Word2Vec
>>> model = Word2Vec.load("300features_40minwords_10context")

2019-06-05 14:26:56,918 : INFO : loading Word2Vec object from 300features_40minwords_10context
2019-06-05 14:26:56,923 : WARNING : this function is deprecated, use smart_open.open instead
2019-06-05 14:26:57,215 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2019-06-05 14:26:57,216 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2019-06-05 14:26:57,219 : INFO : setting ignored attribute vectors_norm to None
2019-06-05 14:26:57,221 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2019-06-05 14:26:57,223 : INFO : setting ignored attribute cum_table to None
2019-06-05 14:26:57,224 : INFO : loaded 300features_40minwords_10context


In [95]:
type(model.wv.syn0 )

/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [96]:
model.wv.syn0.shape

/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(14983, 300)

In [17]:
model["flower"]

/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 3.45470645e-02, -6.92680757e-03, -1.49861231e-01,  2.51579899e-02,
        8.99196565e-02,  4.62126359e-02,  9.31806639e-02,  1.36209959e-02,
        2.38778722e-02,  3.92066091e-02, -4.91487272e-02, -6.64958209e-02,
       -8.20645969e-03,  1.18766315e-01, -7.95729086e-02,  8.31209496e-02,
       -1.40462890e-02, -6.15559854e-02, -7.47416914e-02, -1.30548447e-01,
       -7.18428940e-02,  3.44755277e-02,  6.54032454e-02,  7.32879490e-02,
       -5.84810860e-02, -1.88753521e-03, -6.38130382e-02, -5.36053292e-02,
       -5.17491736e-02,  1.71523506e-03,  4.05737907e-02, -5.72793521e-02,
        4.92551085e-03,  5.54895587e-02, -2.37005390e-02, -1.00924885e-02,
        8.05830359e-02, -1.06857624e-02,  9.06864554e-02,  4.78935540e-02,
        3.61586809e-02,  3.46396677e-02, -2.93709263e-02, -3.36149111e-02,
        5.60174398e-02,  1.58735532e-02, -5.64414598e-02,  3.51388492e-02,
       -7.06660300e-02,  9.27353799e-02,  1.59837473e-02,  5.55619486e-02,
        4.90941703e-02, -

### Averaging the Feature Vectors for the reviews

In [97]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        #
        # Print a status message every 1000th review
        if counter%1000. == 0.:
               print "Review %d of %d" % (counter, len(reviews))
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
        #
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [98]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

print "Cleaning of train reviews"
clean_train_reviews = []
i=0
for review in train["review"]:
    if i%1000. == 0.:
        print "Review %d of %d" % (i, len(train["review"]))
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))
    i+=1

print "Creating average feature vecs for train reviews"
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Cleaning of test reviews"
clean_test_reviews = []
i=0
for review in test["review"]:
    if i%1000. == 0.:
        print "Review %d of %d" % (i, len(train["review"]))
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))
    i+=1

print "Creating average feature vecs for test reviews"
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Cleaning of train reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for train reviews
Review 0 of 25000


/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in divide


Cleaning of test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 1

In [50]:
#cheking the errors\
%debug

> <ipython-input-48-ce9df56b8d21>(46)getAvgFeatureVecs()
     44         #
     45         # Call the function (defined above) that makes average feature vectors
---> 46         reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
     47         #
     48         # Increment the counter

ipdb> print counter
1.0
ipdb> print type(counter)
<type 'float'>

KeyboardInterrupt


### Preprocessing the vectors 

In [99]:
from sklearn.preprocessing import Imputer

trainDataVecs = Imputer().fit_transform(trainDataVecs)
testDataVecs = Imputer().fit_transform(testDataVecs)

/Users/Bai/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/Bai/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### Logistic Regression for Word2Vec

In [100]:
grid_search_w2v = GridSearchCV(log_regression, param_grid, n_jobs=-1, cv=5, scoring="roc_auc")
print("Performing grid search...")

grid_search_w2v.fit(trainDataVecs, train.sentiment)
print("Best CV score: %0.3f" % grid_search_w2v.best_score_)
print("Best parameters set:", grid_search_w2v.best_estimator_.get_params())

Performing grid search...
Best CV score: 0.937
('Best parameters set:', {'warm_start': False, 'C': 1.0, 'n_jobs': None, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'multi_class': 'warn', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'warn', 'class_weight': None})


In [102]:
# Use the random forest to make sentiment label predictions
result = grid_search_w2v.best_estimator_.predict(testDataVecs)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Word2Vec_LinReg_AverageVectors.csv", index=False, quoting=3 )

### Random Forest for Word2Vec

In [73]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


### MLP for Word2Vec

In [121]:
mlp_w2v = mlp.fit(trainDataVecs, train["sentiment"])

# Make sentiment label predictions
result = mlp_w2v.predict(testDataVecs)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Word2Vec_mlp_AverageVectors.csv", index=False, quoting=3 )

## Clustering Word2Vec

In [105]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

/Users/Bai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


Time taken for K Means clustering:  533.072487116 seconds.


In [106]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [107]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'omg']

Cluster 1
[u'devos', u'emmanuelle']

Cluster 2
[u'hue', u'palette']

Cluster 3
[u'plate', u'scratch', u'smoke', u'brake', u'wheel', u'mobile', u'scrap', u'bulldozer', u'coin']

Cluster 4
[u'function', u'communication']

Cluster 5
[u'kaye', u'byron', u'carol', u'hatcher', u'raul', u'daniel', u'fleming', u'darcy', u'colin', u'ross', u'heather', u'graham', u'fiennes']

Cluster 6
[u'biker', u'vegetarian', u'headed']

Cluster 7
[u'iranian', u'experimental']

Cluster 8
[u'plagued']

Cluster 9
[u'enigmatic', u'unsavory', u'crafty', u'oddball', u'eccentric', u'affectionate', u'enthusiastic']


### Creating Bag of Centroids

In [108]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [109]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

### Logistic Regression for Clusters

In [110]:
grid_search_c = GridSearchCV(log_regression, param_grid, n_jobs=-1, cv=5, scoring="roc_auc")
print("Performing grid search...")

grid_search_c.fit(train_centroids, train.sentiment)
print("Best CV score: %0.3f" % grid_search_c.best_score_)
print("Best parameters set:", grid_search_c.best_estimator_.get_params())

Performing grid search...
Best CV score: 0.937
('Best parameters set:', {'warm_start': False, 'C': 0.25, 'n_jobs': None, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'multi_class': 'warn', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'warn', 'class_weight': None})


In [111]:
# Use the random forest to make sentiment label predictions
result = grid_search_c.best_estimator_.predict(test_centroids)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "BagOfCentroids_LinReg.csv", index=False, quoting=3 )

### Random Forest for Clusters

In [83]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


### MLP for Clusters

In [122]:
mlp_c = mlp.fit(train_centroids, train["sentiment"])

# Make sentiment label predictions
result = mlp_c.predict(test_centroids)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "BagOfCentroids_mlp.csv", index=False, quoting=3 )